In [1]:
import warnings
import pandas as pd
from data_analysis.DataProcessor import DataProcessor

warnings.simplefilter(action='ignore', category=FutureWarning)

# read in combined stock data
preprocessor = DataProcessor()
# create df to current date
df = preprocessor.process_nrw_data("06_27")

# ensure timestamp is in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

File already exists: C:\Users\FelixNeubauer\UniRepos\webscraper\data_analysis\processed_nrw_data\06_27_nrw_stock_data.csv. Loading DataFrame from file.


In [2]:
# show columns
df.columns

Index(['id', 'timestamp', 'main_category', 'sub_category', 'product_name',
       'brand', 'price', 'store_skuId', 'store_storeId', 'store_storeName',
       'store_quantity', 'store_availabilityInfo', 'store_clickNcollect1h'],
      dtype='object')

In [3]:
# show all unique values and their count of sub_category, store_storeName and availabilityInfo
sub_category_counts = df['sub_category'].value_counts()
store_name_counts = df['store_storeName'].value_counts()
store_quantity_positive_count = (df['store_quantity'] > 0).sum()

store_quantity_zero_count = (df['store_quantity'] == 0).sum()
print("Unique values and counts for 'sub_category':")
print(sub_category_counts)
print("\nUnique values and counts for 'store_storeName':")
print(store_name_counts)
print("\nCount of entries with 'store_quantity' > 0:", store_quantity_positive_count)
print("Count of entries with 'store_quantity' = 0:", store_quantity_zero_count)

Unique values and counts for 'sub_category':
sub_category
fahrrader                       50643
schlafsacke-matratzen-betten    24586
fitnessgerate-fur-zuhause       23789
campingmobel                    23252
kochen-essen-trinken-camping    21186
zelte                           17192
Name: count, dtype: int64

Unique values and counts for 'store_storeName':
store_storeName
Dortmund-Kampstrasse     13929
Dortmund-Aplerbeck       13754
Herne                    13742
Wuppertal                13682
Köln-Quincy              13663
Essen                    13614
Düsseldorf               13498
Hagen                    13482
Köln Marsdorf            13478
Neuss                    13433
MÜNSTER                  12771
Hückelhoven              11602
Name: count, dtype: Int64

Count of entries with 'store_quantity' > 0: 78521
Count of entries with 'store_quantity' = 0: 82127


## Influence of Stock Outs

The goal of the analysis is to identify an influence of stock outs in neighboring stores on the demand of product variants. A product has an id and store_skuId (the skuId marks i.e. the size or color) - so just the combination makes the product unique (-> product variant). A product variant can be in several stores. So belonging to a store (store Id), just makes the product variant then unique per store. 

The "state" of the quantity of this product variant can change by time in each store (can be found out chronologically by the timestamp). 

A stock out event can be defined as the state of the quantity of a product variant in a store is 0, and was >0 in the measurement before. 

To measure an influence, it's important to only observe variants, which are sold in several stores. We set the minimum number of stores, a product variant has to be sold in, to 5. Further, we only consider product variants with at least 2 stock out events in the observation period across all stores.


In [4]:
# Group by id, store_skuId, and store_storeId to find the number of stores per product variant
product_store_counts = df.groupby(['id', 'store_skuId'])['store_storeId'].nunique().reset_index()
product_store_counts = product_store_counts[product_store_counts['store_storeId'] >= 5]

# Filter the main dataframe to include only those product variants
df_filtered = df.merge(product_store_counts[['id', 'store_skuId']], on=['id', 'store_skuId'])

# Find stock out events
df_filtered['is_stock_out'] = (df_filtered['store_quantity'] == 0) & (df_filtered['store_quantity'].shift() > 0)

# Identify product variants with at least 2 stock out events in different stores
stock_out_events = df_filtered.groupby(['id', 'store_skuId', 'store_storeId'])['is_stock_out'].max().reset_index()
stock_out_counts = stock_out_events.groupby(['id', 'store_skuId'])['is_stock_out'].sum().reset_index()
stock_out_counts = stock_out_counts[stock_out_counts['is_stock_out'] >= 2]

# filter the dataframe to include only those variants
df_relevant_variants  = df_filtered.merge(stock_out_counts[['id', 'store_skuId']], on=['id', 'store_skuId'])

unique_product_variants = df_relevant_variants[['id', 'store_skuId']].drop_duplicates().shape[0]
print("Number of unique product variants after filtering:", unique_product_variants)

Number of unique product variants after filtering: 250


In [6]:
df_relevant_variants.columns


Index(['id', 'timestamp', 'main_category', 'sub_category', 'product_name',
       'brand', 'price', 'store_skuId', 'store_storeId', 'store_storeName',
       'store_quantity', 'store_availabilityInfo', 'store_clickNcollect1h',
       'is_stock_out'],
      dtype='object')

### Enriching with Lag Features and Stock Out Information

In [5]:

# Add stock out information to each state of the product variant
df_relevant_variants['is_stocked_out'] = df_relevant_variants['store_quantity'] == 0

# Convert timestamp to date
df_relevant_variants['date'] = pd.to_datetime(df_relevant_variants['timestamp']).dt.date

# Sort by id, store_skuId, and date
df_relevant_variants = df_relevant_variants.sort_values(['id', 'store_skuId', 'date'])

# Generate lag features for the last 10 days
for lag in range(1, 10):
    df_relevant_variants[f'is_stocked_out_lag_{lag}'] = df_relevant_variants.groupby(['id', 'store_skuId', 'store_storeId'])['is_stocked_out'].shift(lag)

# Define function to calculate daily stock outs and neighboring stock outs directly in the original DataFrame
def calculate_daily_stock_outs(df):
    # TODO 

# Apply the function to calculate daily stock outs and neighboring stock outs
df_relevant_variants = calculate_daily_stock_outs(df_relevant_variants)

# Reset the index for clean DataFrame
df_relevant_variants = df_relevant_variants.reset_index(drop=True)

# Display the resulting DataFrame
print(df_relevant_variants)

               id                  timestamp main_category  \
0             9.0 2024-05-10 19:01:09.860541       fitness   
1             9.0 2024-05-10 19:00:00.010907       fitness   
2             9.0 2024-05-10 19:05:18.320112       fitness   
3             9.0 2024-05-10 18:57:32.676129       fitness   
4             9.0 2024-05-10 19:02:19.139821       fitness   
...           ...                        ...           ...   
85162  25504539.0 2024-06-26 23:50:59.610641       camping   
85163  25504539.0 2024-06-26 23:44:35.262116       camping   
85164  25504539.0 2024-06-26 23:57:19.542666       camping   
85165  25504539.0 2024-06-26 23:54:16.925595       camping   
85166  25504539.0 2024-06-26 23:51:59.178860       camping   

                    sub_category  \
0      fitnessgerate-fur-zuhause   
1      fitnessgerate-fur-zuhause   
2      fitnessgerate-fur-zuhause   
3      fitnessgerate-fur-zuhause   
4      fitnessgerate-fur-zuhause   
...                          ...   
851